In [1]:
using Random
using Distributions

In [2]:
abstract type AbstractBlk end

struct Minor
    label::String
end

struct Genesis <: AbstractBlk end

struct Blk <: AbstractBlk
    index::Int
    minor::Minor
    prev::AbstractBlk
end

function Base.show(io::IO, b::Genesis)
    Base.show(io, 0)
end

function Base.show(io::IO, b::Blk)
    Base.show(io, (b.index, b.minor.label))
end

struct Queue
    sender::Minor
    receiver::Minor
    elem::Vector{AbstractBlk}
    
    Queue(sender, receiver) = new(sender, receiver, Vector{AbstractBlk}())
end

In [3]:
getid(b::Blk) = b.index
getid(b::Genesis) = 0

getid (generic function with 2 methods)

In [4]:
getreceiver(q::Queue) = q.receiver

function receiveblk!(q::Queue, i::Int)::AbstractBlk
    b = q.elem[i]
    deleteat!(q.elem, i)
    b
end

getlength(q::Queue) = length(q.elem)

getlength (generic function with 1 method)

In [5]:
function sendblk!(q::Queue, b::AbstractBlk)
    push!(q.elem, b)
end

sendblk! (generic function with 1 method)

In [6]:
function event_genBlk!(x::Minor, workblk::Dict{Minor,AbstractBlk}, queue::Vector{Queue}, blocks::Vector{Blk})
    prev = workblk[x]
    id = getid(prev) + 1
    b = Blk(id, x, prev)
    push!(blocks, b)
    workblk[x] = b
    for q = queue
        sendblk!(q, b)
    end
    b
end

event_genBlk! (generic function with 1 method)

In [7]:
function event_arrivalBlk!(q::Queue, i::Int, workblk::Dict{Minor,AbstractBlk})
    x = getreceiver(q)
    b = receiveblk!(q, i)
    prev = workblk[x]
    if getid(prev) < getid(b)
        workblk[x] = b
    end
    b
end

event_arrivalBlk! (generic function with 1 method)

In [8]:
function rates(u1, u2, queue, lambda, mu)
#    u1 = sort(collect(keys(lambda)), by=x->x.label)
    r1 = [lambda[x] for x = u1]
#    u2 = sort(collect(keys(mu)), by=x->(x[1].label, x[2].label))
    r2 = [mu[x] * getlength(queue[x]) for x = u2]
    return r1, r2
end

rates (generic function with 1 method)

In [9]:
a = Minor("A")
b = Minor("B")
c = Minor("C")

Minor("C")

In [25]:
lambda = Dict(
    a => 1/10,
    b => 1/5,
    c => 1/10,
)

mu = Dict(
    (a,b) => 1/0.1,
    (a,c) => 1/0.1,
    (b,a) => 1/0.1,
    (b,c) => 1/0.001,
    (c,a) => 1/1,
    (c,b) => 1/0.001,
)

N = 20

20

In [26]:
rng = MersenneTwister(1234)
Aworkblk = []
Ablocks = []
@time for k in 1:N
    blocks = Vector{Blk}()

    events = 0
    u1 = sort(collect(keys(lambda)), by=x->x.label)
    u2 = sort(collect(keys(mu)), by=x->(x[1].label, x[2].label))
    workblk = Dict{Minor,AbstractBlk}(a=>Genesis(), b=>Genesis(), c=>Genesis())
    qAtoB = Queue(a, b)
    qAtoC = Queue(a, c)
    qBtoA = Queue(b, a)
    qBtoC = Queue(b, c)
    qCtoA = Queue(c, a)
    qCtoB = Queue(c, b)
    queue = Dict(
        (a,b)=>qAtoB,
        (a,c)=>qAtoC,
        (b,a)=>qBtoA,
        (b,c)=>qBtoC,
        (c,a)=>qCtoA,
        (c,b)=>qCtoB,
    )
    input = Dict(
        a=>[qAtoB, qAtoC],
        b=>[qBtoA, qBtoC],
        c=>[qCtoA, qCtoB],
    )

    ctime = 0.0
    while ctime < 60*24*30 # 24 hours
        r1, r2 = rates(u1, u2, queue, lambda, mu)
        etime1 = [(r != 0) ? rand(rng, Exponential(1/r)) : Inf for r = r1]
        etime2 = [(r != 0) ? rand(rng, Exponential(1/r)) : Inf for r = r2]
        m1, i1 = findmin(etime1)
        m2, i2 = findmin(etime2)
        if m1 < m2
            ctime += m1
            event_genBlk!(u1[i1], workblk, input[u1[i1]], blocks)
        else
            ctime += m2
            v = rand(rng, DiscreteUniform(1,getlength(queue[u2[i2]])))
            event_arrivalBlk!(queue[u2[i2]], v, workblk)
        end
        events += 1
    end
    push!(Aworkblk,workblk)
    push!(Ablocks,blocks)
end

 11.375778 seconds (42.06 M allocations: 1.313 GiB, 5.84% gc time)


In [27]:
Aworkblk[1]

Dict{Minor,AbstractBlk} with 3 entries:
  Minor("A") => (16956, "C")
  Minor("C") => (16956, "C")
  Minor("B") => (16957, "B")

In [28]:
Achains = []
for i in 1:N
    workblk = Aworkblk[i]
    blk = workblk[a]
    chains = []
    push!(chains, blk)
    while blk != Genesis()
        blk = blk.prev
        push!(chains, blk)
    end
    push!(Achains,chains)
end

In [29]:
Ainvchains = []
for i in 1:N
    chains = Achains[i]
    invchains = reverse(chains)
    push!(Ainvchains,invchains)
end

## 後続にブロックが付かない場合

In [240]:
#blocksにはあるがchainsにないものをカウントする
Ahaki = []
@time for h in 1:N
    haki = 0
    for i in 2:length(Ainvchains[h])
        if (Ainvchains[h][i].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("C"))
            for j in 1:length(Ablocks[h])
                if (Ablocks[h][j].minor == Minor("A")) && (Ablocks[h][j].index == Ainvchains[h][i].index)
                    haki += 1
                end
            end
        end
    end
   push!(Ahaki,haki)
    if h%20 == 0
        print(h,"ok")
    end
end

20ok1202.703518 seconds (19.65 G allocations: 358.926 GiB, 10.01% gc time)


## 後続にブロックが1つ付いた場合

In [30]:
#blocksで連番かつchainsにない場合は二つ以上同時に破棄されていると考える
Ahaki = []
@time for h in 1:N
    haki = 0
    count = 0
    for i in 3:length(Ainvchains[h])-1
        if i%1000 == 0
            count += 1
            print(count,",")
        end
#        if Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i-1].minor == Minor("B")
#            for j in 1:length(Ablocks[h])
#                if (Ablocks[h][j].minor == Minor("A")) && (Ablocks[h][j].index == Ainvchains[h][i-1].index)
#                    for k in 1:length(Ablocks[h])
#                        if (Ablocks[h][k].minor == Minor("A")) && (Ablocks[h][k].index == Ainvchains[h][i].index)
#                            haki += 1
#                        end
#                    end
#                end
#            end
        if (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+1].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+1].minor == Minor("C")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+1].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+1].minor == Minor("C"))
            for j in 1:length(Ablocks[h])
                if (Ablocks[h][j].minor == Minor("A")) && (Ablocks[h][j].index == Ainvchains[h][i].index)
                    for k in 1:length(Ablocks[h])
                        if (Ablocks[h][k].minor == Minor("A")) && (Ablocks[h][k].index == Ainvchains[h][i+1].index)
                            haki += 1
                        end
                    end
                end
            end
        end
    end
    push!(Ahaki,haki)
    if h%20 == 0
        print(h,"ok")
    end
end

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,20ok917.199156 seconds (15.86 G allocations: 289.638 GiB, 8.38% gc time)


## 後続にブロックが2つ付いた場合

In [271]:
Ahaki = []
@time for h in 1:N
    haki = 0
    count = 0
    for i in 3:length(Ainvchains[h])-1
        if i%1000 == 0
            count += 1
            print(count,",")
        end
        
        if (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+1].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+1].minor == Minor("C")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+1].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+1].minor == Minor("C"))
            if (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+2].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+2].minor == Minor("C")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+2].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+2].minor == Minor("C"))
                for j in 1:length(Ablocks[h])
                    if (Ablocks[h][j].minor == Minor("A")) && (Ablocks[h][j].index == Ainvchains[h][i].index)
                        for k in 1:length(Ablocks[h])
                            if (Ablocks[h][k].minor == Minor("A")) && (Ablocks[h][k].index == Ainvchains[h][i+1].index)
                                if (Ablocks[h][k].minor == Minor("A")) && (Ablocks[h][k].index == Ainvchains[h][i+2].index)
                                    haki += 1
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    push!(Ahaki,haki)
    if h%20 == 0
        print(h,"ok")
    end
end

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,

BoundsError: BoundsError: attempt to access 17069-element Array{Any,1} at index [17070]

In [31]:
AblkA = []
for i in 1:N
    blkA = 0
    for j in 1:length(Ablocks[i])
        if Ablocks[i][j].minor == Minor("A")
            blkA += 1
        end
    end
    push!(AblkA,blkA)
end

In [32]:
Aresult = []
for i in 1:N
    result = Ahaki[i] / AblkA[i]
    push!(Aresult,result)
end

## 平均

In [33]:
sum = 0
for i in 1:N
    sum += Aresult[i]
end
ave = sum / N

0.0024831696506798133

## 分散

In [34]:
sq = 0
for i in 1:N
    sq += (Aresult[i]-ave)^2
end
var = sq / N

5.91540207512151e-7

In [35]:
sta = sqrt(var)

0.0007691165110125715

## 信頼区間(95%)

In [36]:
t = 2.0860

2.086

In [37]:
sita = ave - t * (1/10) * sta
ue = ave + t * (1/10) * sta
print("[",sita,",",ue,"]")

[0.002322731946482591,0.002643607354877036]

In [38]:
e = 0
for i in 1:N
    if Aresult[i] < sita && Aresult[i] > ue
        e += 1
    end
end
e

0

In [39]:
Ahaki

20-element Array{Any,1}:
 10
 12
  8
  5
  9
 13
  9
 11
 11
 15
 10
 14
  8
 12
 17
  3
  8
 11
 14
 15

In [18]:
open("chains.txt","w") do out #ファイルに書く
    print(out,chains)
end

In [19]:
open("blocks.txt","w") do out #ファイルに書く
    print(out,blocks)
end